In [4]:
import numpy as np
import scipy.io as sio
import scipy.signal as sps
import robustsp as rsp
import matplotlib.pyplot as plt
import matplotlib

p=1
q=0
x = sio.loadmat('x')['x'].flatten()
x_ao = sio.loadmat('x_ao')['x_ao'].flatten() # max(x) = 4.6821

# snippet from arma_est_bip_s that utilizes bip_s_resid_sc 
from scipy.optimize import least_squares as lsq
beta_initial = rsp.robust_starting_point(x_ao,p,q)[0] # stimmt

F_bip = lambda beta: rsp.bip_s_resid_sc(x_ao, beta, p, q)[2] if len(beta)!=1 else lambda beta: rsp.bip_s_resid_sc(x_ao, beta, p, q)[0]
beta_bip = lsq(F_bip, beta_initial,xtol=5*1e-7,ftol=5*1e-7,method='lm')['x'] # should -.7811
a,xf,c = rsp.bip_s_resid_sc(x_ao,beta_bip,p,q)
 
print(beta_bip,[-.7811])
print(a, rsp.bip_s_resid_sc(x_ao, [-.7811], p, q)[0])
np.max(xf), np.max(rsp.bip_s_resid_sc(x_ao, [-.7811], p, q)[1]),np.max(x)

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [6]:
np.max(rsp.bip_s_resid_sc(x, beta_bip, p, q)[1]), np.max(x), np.max(rsp.bip_s_resid_sc(x_ao, [-.7811], p, q)[1])

(2.534609372401701, 4.682137999981019, 4.769117471003652)

In [9]:
res0=rsp.arma_est_bip_s(x_ao,p,q) # x_filt from rsp.bip_s_resid_sc(x, beta_bip, p, q)[1]
np.max(res0['cleaned_signal']), np.max(x)

(0.0, 4.682137999981019)

In [8]:
result = rsp.arma_est_bip_mm(x_ao,p,q) # x_filt from rsp.arma_est_bip_s(x,p,q)
np.max(result['cleaned_signal']), np.max(x)

(0.0, 4.682137999981019)

In [8]:
beta_bip, np.max(rsp.bip_s_resid_sc(x_ao,[-.75], p, q)[1]) ,np.max(rsp.bip_s_resid_sc(x_ao, beta_bip, p, q)[1]) # should be ~4.7

(array([1.39002852]), 4.769117471003652, 275.1901935905804)

In [11]:
phi_hat = np.array(beta_initial[:p])
theta_hat = np.array(beta_initial[p:])
N = len(x_ao)
r = max(p,q)

a_bip = np.zeros(N)
kap2 = 0.8724286
x_sc = rsp.m_scale(x_ao)
lamb = rsp.ma_infinity(phi_hat, -1*theta_hat, 100)
sigma_hat = np.sqrt(x_sc**2/(1+kap2*np.sum(lamb**2)))
# AR Model
for ii in range(r,N):
    xArr = x[ii-1::-1] if ii-p-1 < 0 else x[ii-1:ii-p-1:-1]
    apArr = a_bip[ii-1::-1] if ii-p-1 < 0 else a_bip[ii-1:ii-p-1:-1]

    a_bip[ii] = x[ii]-phi_hat@(xArr-apArr\
    +sigma_hat*rsp.eta(apArr/sigma_hat))


a_bip_sc = rsp.m_scale(a_bip[p:])
   
a_bip_sc, rsp.bip_s_resid_sc(x_ao, beta_initial, p, q)[0]

(0.9102497169668862, 1.86631033110071)

In [7]:
beta_hat = beta_initial[:]

phi_hat = np.array(beta_hat[:p]) # [] if p=0, MA case

theta_hat = np.array(beta_hat[p:]) # [] if p=length(beta_hat), AR case

#x = np.array(x)
N = len(x)
r = max(p,q)

a_bip = np.zeros(N)

x_sc = rsp.m_scale(x)

kap2 = 0.8724286

poles = lambda x: np.sum(np.abs(np.roots(-1*np.array([-1,*x]))) >1)

if poles(phi_hat) or poles(theta_hat):
    sigma_hat = x_sc   
else:
    lamb = rsp.ma_infinity(phi_hat, -1*theta_hat, 100) # MA infinity approximation to compute scale used in eta function
    sigma_hat = np.sqrt(x_sc**2/(1+kap2*np.sum(lamb**2))) # scale used in eta

if r == 0:
    a_sc_bip = x_sc
    a_bip = np.array(x)
else:
    if poles(phi_hat) or poles(theta_hat):
        pass
        #return 10**10,a_bip,a_bip[p:]
    elif p>=1 and q>=1:
        # ARMA model
        for ii in range(r,N):
            # BIP-ARMA residuals
            xArr = x[ii-1::-1] if ii-p-1 < 0 else x[ii-1:ii-p-1:-1]
            aArr = a_bip[ii-1::-1] if ii-q-1 < 0 else a_bip[ii-1:ii-q-1:-1]
            apArr = a_bip[ii-1::-1] if ii-p-1 < 0 else a_bip[ii-1:ii-p-1:-1]

            a_bip[ii] = x[ii]\
            -phi_hat@(xArr-apArr+sigma_hat*rsp.eta(apArr/sigma_hat))+sigma_hat*(theta_hat[None,:])@(rsp.eta(aArr/sigma_hat)[:,None])

    elif p==0 and q>=1:
        # MA residuals
        for ii in range(r,N):
            # BIP-MA residuals
            aArr = a_bip[ii-1::-1] if ii-q-1 < 0 else a_bip[ii-1:ii-q-1:-1]

            a_bip[ii] = x[ii]+theta_hat[None,:]@(sigma_hat*rsp.eta(aArr/sigma_hat)[:,None])

    elif p>=1 and q==0:
        # AR Model
        for ii in range(r,N):
            xArr = x[ii-1::-1] if ii-p-1 < 0 else x[ii-1:ii-p-1:-1]
            apArr = a_bip[ii-1::-1] if ii-p-1 < 0 else a_bip[ii-1:ii-p-1:-1]

            a_bip[ii] = x[ii]-phi_hat@(xArr-apArr\
            +sigma_hat*rsp.eta(apArr/sigma_hat))


    a_bip_sc = rsp.m_scale(a_bip[p:])
    x_filt = np.array(x)

    for ii in range(p,N):
        x_filt[ii] = x[ii] - a_bip[ii] + sigma_hat * rsp.eta(a_bip[ii]/sigma_hat)

a_bip_sc, np.max(rsp.bip_s_resid_sc(x_ao, beta_initial, p, q)[1]), np.max(x)

(0.9179389286960025, 4.769117471003652, 4.682137999981019)